<a href="https://colab.research.google.com/github/MichalBalcerak/ML24-25/blob/main/Homework06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equivalence Analysis
### Are *local RSS minimization* and *RSS gain maximization* equivalent?


Consider the simple dataset with one feature $X$ and labels $y$:
$$
\begin{array}{|c|c|c|}
\hline
\text{X} & \text{y} \\
\hline
1 & 1  \\
2 & 1 \\
3 & 6  \\
4 & 8  \\
5 & 10  \\
\hline
\end{array}
$$

**The first step** of both methods is the same - split data into 2 parts, dependent on value of $X$. At the start there is the only one node to split, so we calculate total RSS and this is equivalent to local RSS. As the *parent RSS* is constant, due to one node, *RSS gain* maximization is equivalent to *local RSS* minimization.

The threshold is $X < 3$. Then RSS $\sum_m\sum_i(y_i-\overline{y_{R_m}})^2$ is minimal. Now we have:

*  Left leaf: $X < 3$: $x\in\{1,2\},\quad \overline{y}=1,\quad\sum_i(y_i-\overline{y})^2=0$
*  Right leaf: $X \geq 3$: $x\in\{3,4,5\},\quad \overline{y}=\frac{6+8+10}{3}=8,\quad\sum_i(y_i-\overline{y})^2=(6-8)^2+(10-8)^2=8$

But **the second step is different**:

**If we split left leaf** (by the condition $X< 2$), we get two new leaves, each with the *local RSS* equals to $0$, because both of them will have one element ($x\in\{1\}$ and $x\in\{2\}$).

**If we split right leaf**, the condition is $X< 4$ (without loss of generality - the condition $X< 5$ - provides the same local RSS value), we get two new leaves:


*   *(child)* Left leaf: $X < 5$: $x\in\{3\}, \quad \overline{y}=6,\quad\sum_i(y_i-\overline{y})^2=0$
*  *(child)* Right leaf: $X \geq 5$: $x\in\{4,5\}, \quad \overline{y}=\frac{8+10}{2},\quad\sum_i(y_i-\overline{y})^2=2$

Since *local RSS* equals to $0$ in the first case (splitting the left leaf) and equals to $0+2=2$ in the second case (splitting the right leaf), **Local RSS Minimization algorithm selects the first case**.

However, if we split the left leaf, reduction in RSS equals to $0$. If we split the right leaf, it is equal to $8-2=6$. Therefore, **RSS Gain Maximization algorithm selects the second case** because reduction in RSS is greater in that case.

Thus, *local RSS minimization* and *RSS gain maximization* are **not** equivalent.


### Does *total RSS minimization* coincide with either of these two, or is it distinct?

*Total RSS minimization* **is equivalent to** *RSS Gain Maximization*

Proof:

Suppose we chose feature $X$, threshold $t$ and split leaf $M$.

Let's notice that *total RSS value* is (after that split) equals to:
$$\sum_{m\in\textit{Leaves}(T(X,t))}\sum_{i\in R_m}(y_i-\overline{y_{R_m}})^2=\sum_{m\in\textit{Leaves}(T(X,t))\backslash\{M\}}\sum_{i\in R_m}(y_i-\overline{y_{R_m}})^2-\underbrace{\left(\sum_{i\in R_M}(y_i-\overline{y_{R_M}})^2 - \sum_{m=1}^2\sum_{i\in R_m}(y_i-\overline{y_{R_m}})^2\right)}_{\text{Parent RSS - Children RSS = RSS Gain}}$$
*All notations are similar to the ones were used in class. Note that $\textit{Leaves}(T(X,t))\backslash\{M\}$ is the set of all leaves but without selected $M$ leaf.*

As we can see, **minimization of *total RSS value* is equivalent to *maximization of RSS Gain* ,** since the term
$$\sum_{m\in\textit{Leaves}(T(X,t))\backslash\{M\}}\sum_{i\in R_m}(y_i-\overline{y_{R_m}})^2$$
is constant in both methods- this is the sum of RSS in all leaves except $M$ - the one which was split, which means that methods *Total RSS minimization* and *RSS Gain Maximization* are equivalent.
